<a href="https://colab.research.google.com/github/auzaluis/upsa_mod_202501/blob/main/personalidad/01_script_ETL_personalidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 01: Carga de datos

## Importando base de datos

In [1]:
# Google Auth
from google.colab import auth
auth.authenticate_user()

In [2]:
# API client
from google.auth import default
creds, _ = default()

In [3]:
# gspread authorization
import gspread
gc = gspread.authorize(creds)

In [4]:
# Accediendo al Google Sheet
url_personalidad = 'https://docs.google.com/spreadsheets/d/1IQ_RxxTSmBKHTExlxboIRNlMov_F6RyqdcOPrflCv_w/edit?usp=sharing'
gsheets = gc.open_by_url(url_personalidad)
sheets = gsheets.worksheet('Respuestas de formulario 1').get_all_values()

In [5]:
type(sheets)

list

In [6]:
# Convirtiendo la lista en df
import pandas as pd
df = pd.DataFrame(sheets[1:], columns=sheets[0])

## Inspección del data frame

In [7]:
type(df)

pandas.core.frame.DataFrame

In [8]:
# ver el tamaño del df
df.shape

(245, 33)

In [9]:
# Ver las primeras filas
df.head()

,Marca temporal,"¿Estás estudiando en algún colegio, universidad o instituto?",Escribe tu edad exacta,Sexo,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me preocupa mucho el medio ambiente],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Estudio mucho, doy mi mayor esfuerzo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco el éxito sin importar lo que deba sacrificar],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Trato de vestir sencillo para no presumir],...,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta conocer gente y hacer nuevos amigos],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy una persona casera, prefiero estar en casa haciendo lo mío]","Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Prefiero hacer las cosas solo, me estresa trabajar en grupo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta tener muchos amigos y pasar tiempo con ellos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me cuesta concentrarme porque me aburro rápido de las cosas],Cuando tenga 30 años me gustaría:,¿Cuánto tiempo has pasado en los últimos 7 días en TikTok? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Instagram? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Facebook? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en YouTube? Si no lo las usado coloca 00:00
0,9/08/2021 12:23:46,Sí,16,Hombre,No lo sé,No lo sé,No lo sé,Totalmente verdadero,Un poco verdadero,No lo sé,...,Un poco verdadero,Un poco verdadero,Un poco verdadero,Un poco verdadero,No lo sé,"Formar una familia, pues me daría una motivaci...",0:00:00,1:00:00,1:00:00,2:00:00
1,9/08/2021 12:25:01,Sí,16,Mujer,No lo sé,Un poco verdadero,Totalmente verdadero,Un poco falso,Un poco falso,Totalmente falso,...,Un poco verdadero,Un poco verdadero,Totalmente verdadero,No lo sé,Totalmente falso,"Formar una familia, pues me daría una motivaci...",0:50:00,0:00:00,0:00:00,1:00:00
2,9/08/2021 12:26:31,Sí,17,Hombre,Totalmente verdadero,Totalmente falso,No lo sé,Un poco verdadero,Un poco verdadero,Totalmente verdadero,...,Un poco verdadero,No lo sé,No lo sé,Un poco verdadero,No lo sé,"Formar una familia, pues me daría una motivaci...",8:00:00,4:00:00,0:00:00,8:00:00
3,9/08/2021 12:28:49,Sí,22,Mujer,Totalmente verdadero,Totalmente falso,Un poco verdadero,Totalmente verdadero,Un poco verdadero,Un poco falso,...,Un poco verdadero,Un poco verdadero,Totalmente verdadero,Un poco verdadero,Un poco falso,"Formar una familia, pues me daría una motivaci...",22:35:00,11:21:00,1:57:00,2:40:00
4,9/08/2021 12:31:43,Sí,16,Hombre,Totalmente verdadero,Un poco verdadero,Un poco falso,Totalmente falso,Un poco falso,Un poco falso,...,Un poco verdadero,Un poco falso,Un poco falso,Totalmente verdadero,Un poco verdadero,"Estar soltero, muy enfocado en mis logros pers...",7:00:00,1:04:00,0:00:00,3:26:00


In [10]:
# Ver al estructura del df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 33 columns):
 #   Column                                                                                                                                                   Non-Null Count  Dtype 
---  ------                                                                                                                                                   --------------  ----- 
 0   Marca temporal                                                                                                                                           245 non-null    object
 1   ¿Estás estudiando en algún colegio, universidad o instituto?                                                                                             245 non-null    object
 2   Escribe tu edad exacta                                                                                                                                   245 non-null 

In [11]:
# dtype
df['Sexo'].dtype

dtype('O')

In [12]:
# Número de filas
len(df)

245

In [13]:
# Número de columnas
len(df.columns)

33

# Tema 02: Transformación de datos

## Valores perdidos

### Identificando NAs

In [14]:
# Los NAs están como strings vacíos
import numpy as np
df.replace('', np.nan, inplace=True)

In [15]:
df['Escribe tu edad exacta'] \
.isna() \
.value_counts()

,count
Escribe tu edad exacta,
False,239
True,6


### Tratando los NAs

In [16]:
df['Escribe tu edad exacta'].dtype

dtype('O')

In [17]:
df['Escribe tu edad exacta'] = pd.to_numeric(df['Escribe tu edad exacta'], errors='coerce')

In [18]:
df['Escribe tu edad exacta'].dtype

dtype('float64')

Reemplazo por la media

In [19]:
edad_promedio = df['Escribe tu edad exacta'].mean()

In [20]:
# Creando df2
df2 = df.copy()

In [21]:
# Reemplazo por la media
df2['edad2'] = df2['Escribe tu edad exacta'].fillna(edad_promedio)

In [22]:
df2[['Escribe tu edad exacta', 'edad2']]

,Escribe tu edad exacta,edad2
0,16.0,16.0
1,16.0,16.0
2,17.0,17.0
3,22.0,22.0
4,16.0,16.0
...,...,...
240,22.0,22.0
241,21.0,21.0
242,22.0,22.0
243,21.0,21.0


Reubicando edad2

In [23]:
# Crear una lista con los nombres de las columnas
lista_columnas = list(df2.columns)

In [24]:
# Quitando temporalmente edad2 de la lista
lista_columnas.remove('edad2')

In [25]:
# Obteniendo el índice (posición) de la columna "after"
lista_columnas.index('Escribe tu edad exacta')

2

In [26]:
# Insertando "edad2" después de "Escribe tu edad exacta"
lista_columnas.insert(lista_columnas.index('Escribe tu edad exacta') + 1, 'edad2')

In [27]:
df2 = df2[lista_columnas]

In [28]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 34 columns):
 #   Column                                                                                                                                                   Non-Null Count  Dtype  
---  ------                                                                                                                                                   --------------  -----  
 0   Marca temporal                                                                                                                                           245 non-null    object 
 1   ¿Estás estudiando en algún colegio, universidad o instituto?                                                                                             245 non-null    object 
 2   Escribe tu edad exacta                                                                                                                                   239 non-n

Creando relocate() en py

In [29]:
def relocate(df, columna, after):
  lista_columnas = list(df.columns)
  lista_columnas.remove(columna)
  lista_columnas.insert(lista_columnas.index(after) + 1, columna)
  return df[lista_columnas]

In [30]:
relocate(
    df = df2,
    columna = 'edad2',
    after = 'Escribe tu edad exacta'
)

,Marca temporal,"¿Estás estudiando en algún colegio, universidad o instituto?",Escribe tu edad exacta,edad2,Sexo,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me preocupa mucho el medio ambiente],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Estudio mucho, doy mi mayor esfuerzo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco el éxito sin importar lo que deba sacrificar],...,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta conocer gente y hacer nuevos amigos],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy una persona casera, prefiero estar en casa haciendo lo mío]","Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Prefiero hacer las cosas solo, me estresa trabajar en grupo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta tener muchos amigos y pasar tiempo con ellos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me cuesta concentrarme porque me aburro rápido de las cosas],Cuando tenga 30 años me gustaría:,¿Cuánto tiempo has pasado en los últimos 7 días en TikTok? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Instagram? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Facebook? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en YouTube? Si no lo las usado coloca 00:00
0,9/08/2021 12:23:46,Sí,16.0,16.0,Hombre,No lo sé,No lo sé,No lo sé,Totalmente verdadero,Un poco verdadero,...,Un poco verdadero,Un poco verdadero,Un poco verdadero,Un poco verdadero,No lo sé,"Formar una familia, pues me daría una motivaci...",0:00:00,1:00:00,1:00:00,2:00:00
1,9/08/2021 12:25:01,Sí,16.0,16.0,Mujer,No lo sé,Un poco verdadero,Totalmente verdadero,Un poco falso,Un poco falso,...,Un poco verdadero,Un poco verdadero,Totalmente verdadero,No lo sé,Totalmente falso,"Formar una familia, pues me daría una motivaci...",0:50:00,0:00:00,0:00:00,1:00:00
2,9/08/2021 12:26:31,Sí,17.0,17.0,Hombre,Totalmente verdadero,Totalmente falso,No lo sé,Un poco verdadero,Un poco verdadero,...,Un poco verdadero,No lo sé,No lo sé,Un poco verdadero,No lo sé,"Formar una familia, pues me daría una motivaci...",8:00:00,4:00:00,0:00:00,8:00:00
3,9/08/2021 12:28:49,Sí,22.0,22.0,Mujer,Totalmente verdadero,Totalmente falso,Un poco verdadero,Totalmente verdadero,Un poco verdadero,...,Un poco verdadero,Un poco verdadero,Totalmente verdadero,Un poco verdadero,Un poco falso,"Formar una familia, pues me daría una motivaci...",22:35:00,11:21:00,1:57:00,2:40:00
4,9/08/2021 12:31:43,Sí,16.0,16.0,Hombre,Totalmente verdadero,Un poco verdadero,Un poco falso,Totalmente falso,Un poco falso,...,Un poco verdadero,Un poco falso,Un poco falso,Totalmente verdadero,Un poco verdadero,"Estar soltero, muy enfocado en mis logros pers...",7:00:00,1:04:00,0:00:00,3:26:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,8/02/2025 10:43:39,Sí,22.0,22.0,Hombre,Un poco verdadero,Un poco verdadero,Un poco verdadero,Un poco verdadero,Un poco verdadero,...,Un poco falso,Totalmente verdadero,No lo sé,No lo sé,Un poco verdadero,"Estar soltero, muy enfocado en mis logros pers...",0:00:00,2:54:00,0:00:00,5:13:00
241,8/02/2025 10:44:18,Sí,21.0,21.0,Mujer,No lo sé,Un poco falso,No lo sé,Un poco verdadero,No lo sé,...,Un poco verdadero,Un poco falso,Un poco falso,Un poco verdadero,No lo sé,"Formar una familia, pues me daría una motivaci...",13:00:00,3:25:00,0:39:00,0:00:00
242,8/02/2025 10:44:33,Sí,22.0,22.0,Mujer,Totalmente verdadero,Un poco verdadero,Un poco verdadero,Totalmente verdadero,Totalmente verdadero,...

Eliminar la fila completa

In [31]:
df2.shape

(245, 34)

In [32]:
df2.dropna(inplace=True)

In [33]:
df2.shape

(239, 34)

## Estandarización de variables

### Normalización

In [34]:
from sklearn.preprocessing import StandardScaler

In [35]:
# instansciando StandardScaler()
normalizador = StandardScaler()

In [36]:
# normalizando
normalizador.fit_transform(df2[['edad2']])

array([[-1.4849251 ],
       [-1.4849251 ],
       [-1.23106451],
       [ 0.03823841],
       [-1.4849251 ],
       [-0.97720393],
       [-1.99264627],
       [-0.46948276],
       [ 0.03823841],
       [-0.21562217],
       [-1.73878568],
       [ 0.03823841],
       [-0.21562217],
       [-0.97720393],
       [-0.97720393],
       [-1.73878568],
       [-1.4849251 ],
       [-1.73878568],
       [-0.97720393],
       [ 0.292099  ],
       [-0.46948276],
       [-1.73878568],
       [ 0.79982017],
       [ 0.03823841],
       [-1.23106451],
       [-0.97720393],
       [-0.72334334],
       [-0.72334334],
       [-0.97720393],
       [-0.97720393],
       [-0.46948276],
       [-1.73878568],
       [-0.21562217],
       [-0.21562217],
       [-0.46948276],
       [-0.46948276],
       [-0.46948276],
       [-0.46948276],
       [-0.46948276],
       [ 0.292099  ],
       [-0.21562217],
       [ 0.292099  ],
       [-0.21562217],
       [ 0.03823841],
       [-0.21562217],
       [ 0

In [37]:
# creando df3
df3 = df2.copy()

In [38]:
df3['edadZ'] = normalizador.fit_transform(df3[['edad2']])

In [39]:
df3 = relocate(
  df = df3,
  columna = 'edadZ',
  after = 'edad2'
)

### Rango

In [40]:
from sklearn.preprocessing import MinMaxScaler

In [41]:
# instanciando MinMaxScaler()
rango = MinMaxScaler()

In [42]:
df3['edadR'] = rango.fit_transform(df3[['edad2']])

In [43]:
df3 = relocate(
  df = df3,
  columna = 'edadR',
  after = 'edadZ'
)

In [44]:
df3[['edad2', 'edadZ', 'edadR']]

,edad2,edadZ,edadR
0,16.0,-1.484925,0.058824
1,16.0,-1.484925,0.058824
2,17.0,-1.231065,0.088235
3,22.0,0.038238,0.235294
4,16.0,-1.484925,0.058824
...,...,...,...
240,22.0,0.038238,0.235294
241,21.0,-0.215622,0.205882
242,22.0,0.038238,0.235294
243,21.0,-0.215622,0.205882


## Agrupaciones

### Numéricas

In [45]:
cortes = [-float('inf'), 18, 21, float('inf')]
etiquetas = ['18 o menos', '19 a 21', 'Más de 21']

In [46]:
df3['edadGR'] = pd.cut(
    x = df3['edad2'],
    bins = cortes,
    labels = etiquetas
)

In [47]:
df3 = relocate(
    df = df3,
    columna = 'edadGR',
    after = 'edadR'
)

In [48]:
df3['edadGR'].value_counts()

,count
edadGR,
Más de 21,111
19 a 21,97
18 o menos,31


### Categóricas

In [49]:
# Revisando índices
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 239 entries, 0 to 244
Data columns (total 37 columns):
 #   Column                                                                                                                                                   Non-Null Count  Dtype   
---  ------                                                                                                                                                   --------------  -----   
 0   Marca temporal                                                                                                                                           239 non-null    object  
 1   ¿Estás estudiando en algún colegio, universidad o instituto?                                                                                             239 non-null    object  
 2   Escribe tu edad exacta                                                                                                                                   239 non-nu

In [50]:
# Valores únicos
pd.unique(df3.iloc[:,8])

array(['No lo sé', 'Totalmente verdadero', 'Un poco falso',
       'Un poco verdadero', 'Totalmente falso'], dtype=object)

In [51]:
# Frecuencias
df3.iloc[:,8].value_counts()

,count
Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],
Totalmente verdadero,132
Un poco verdadero,65
No lo sé,28
Un poco falso,12
Totalmente falso,2


In [52]:
df3.iloc[:,8].isin(['Totalmente verdadero', 'Un poco verdadero'])

,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual]
0,False
1,False
2,True
3,True
4,True
...,...
240,True
241,False
242,True
243,True


Dummitización

In [53]:
# Función condicional normal
def top2box(x):
  # if x == 'Totalmente verdadero' or x == 'Un poco verdadero':
  if x in ['Totalmente verdadero', 'Un poco verdadero']:
    return 1
  else:
    return 0

In [54]:
# Aplicando la función
df3.iloc[:,8] \
.apply(top2box) \
.value_counts()

,count
Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],
1,197
0,42


In [55]:
# Función lambda
df3.iloc[:,8].apply(
    lambda x: 1 if x in ['Totalmente verdadero', 'Un poco verdadero'] else 0
)

,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual]
0,0
1,0
2,1
3,1
4,1
...,...
240,1
241,0
242,1
243,1


In [56]:
# Frecuencias
df3.iloc[:,8] \
.apply(lambda x: 1 if x in ['Totalmente verdadero', 'Un poco verdadero'] else 0) \
.value_counts()

,count
Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],
1,197
0,42


### Ordinales

In [57]:
import numpy as np

In [58]:
pd.unique(df3.iloc[:,8])

array(['No lo sé', 'Totalmente verdadero', 'Un poco falso',
       'Un poco verdadero', 'Totalmente falso'], dtype=object)

In [59]:
# Cuando las codiciones son muchas (5 en esta caso), se recomienda usar select()
condiciones = [
    df3.iloc[:,8] == 'Totalmente verdadero',
    df3.iloc[:,8] == 'Un poco verdadero',
    df3.iloc[:,8] == 'No lo sé',
    df3.iloc[:,8] == 'Un poco falso',
    df3.iloc[:,8] == 'Totalmente falso',
]

# Hacer coincidir el orden
opciones = [5,4,3,2,1]

In [60]:
# Usando select()
ordinal = np.select(
    condlist = condiciones,
    choicelist = opciones,
    default = np.nan
)

In [61]:
ordinal

array([3., 3., 5., 5., 5., 5., 5., 5., 3., 2., 5., 3., 4., 5., 4., 5., 5.,
       4., 2., 4., 2., 5., 5., 2., 5., 4., 5., 4., 5., 3., 5., 5., 5., 4.,
       5., 5., 4., 5., 5., 2., 3., 5., 5., 5., 5., 4., 5., 5., 5., 5., 5.,
       5., 5., 3., 4., 4., 5., 5., 5., 4., 5., 5., 5., 5., 5., 3., 5., 5.,
       4., 5., 4., 5., 4., 3., 4., 4., 4., 5., 4., 5., 5., 5., 4., 4., 4.,
       2., 5., 3., 5., 4., 3., 4., 3., 5., 5., 4., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 4., 5., 5., 5., 5., 5., 4., 5., 5., 5., 5., 5., 5., 5.,
       4., 5., 4., 4., 5., 4., 5., 4., 5., 5., 5., 5., 5., 5., 3., 5., 5.,
       4., 5., 4., 3., 5., 4., 4., 5., 4., 5., 5., 3., 4., 5., 4., 5., 5.,
       5., 4., 5., 4., 4., 4., 5., 5., 4., 5., 5., 4., 4., 5., 5., 3., 5.,
       5., 4., 5., 5., 5., 5., 5., 5., 5., 5., 5., 3., 4., 2., 1., 4., 5.,
       3., 3., 2., 5., 4., 4., 3., 2., 5., 3., 4., 3., 2., 5., 3., 4., 5.,
       5., 4., 3., 5., 4., 5., 2., 5., 5., 1., 4., 4., 3., 5., 3., 5., 4.,
       5., 4., 5., 5., 3.

In [62]:
ordinal.mean()

np.float64(4.309623430962343)

In [63]:
np.unique(ordinal, return_counts = True)

(array([1., 2., 3., 4., 5.]), array([  2,  12,  28,  65, 132]))

### Bucles

Forma standard

In [64]:
# creando df4
df4 = df3.copy()

In [65]:
df4.columns

Index(['Marca temporal',
       '¿Estás estudiando en algún colegio, universidad o instituto?',
       'Escribe tu edad exacta', 'edad2', 'edadZ', 'edadR', 'edadGR', 'Sexo',
       'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual]',
       'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status]',
       'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me preocupa mucho el medio ambiente]',
       'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Estudio mucho, doy mi mayor esfuerzo]',
       'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco el éxito sin importar lo que deba sacrificar]',
       'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Trato de vestir sencillo para no presumir]',
       'Según tu forma de ser ¿Cuál de las siguientes frases te de

In [66]:
# crear una lista vacía
frases = []

# se ejecuta el bucle para llenar la lista
for col in df4.columns:
  if col.startswith('Según tu'):
    frases.append(col)

In [67]:
frases

['Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual]',
 'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status]',
 'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me preocupa mucho el medio ambiente]',
 'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Estudio mucho, doy mi mayor esfuerzo]',
 'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco el éxito sin importar lo que deba sacrificar]',
 'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Trato de vestir sencillo para no presumir]',
 'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Creo que la vida se trata de tomar riesgos]',
 'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco hacer cosas emocionantes para no aburrirme]',
 'Según tu forma de ser ¿Cuá

In [68]:
# aplicando el bucle para dummitizar, recordemos que creamos la función top2box()
for frase in frases:
  df4[frase] = df4[frase].apply(top2box)

In [69]:
df4[frases]

,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me preocupa mucho el medio ambiente],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Estudio mucho, doy mi mayor esfuerzo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco el éxito sin importar lo que deba sacrificar],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Trato de vestir sencillo para no presumir],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Creo que la vida se trata de tomar riesgos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco hacer cosas emocionantes para no aburrirme],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [En mi casa, la familia es muy importante, por eso paso mucho tiempo con ellos]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Tener dinero es clave para ser respetado],...,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comer sano y estar en forma],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Vestirse bien es clave para el éxito ],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy sincero con las personas, aunque la verdad les duela]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me preocupo y ayudo a los necesitados],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Invierto mucho tiempo y dinero para aprender cosas nuevas],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta conocer gente y hacer nuevos amigos],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy una persona casera, prefiero estar en casa haciendo lo mío]","Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Prefiero hacer las cosas solo, me estresa trabajar en grupo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta tener muchos amigos y pasar tiempo con ellos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me cuesta concentrarme porque me aburro rápido de las cosas]
0,0,0,0,1,1,0,1,0,1,0,...,1,1,1,1,1,1,1,1,1,0
1,0,1,1,0,0,0,1,1,1,0,...,0,1,1,1,0,1,1,1,0,0
2,1,0,0,1,1,1,0,1,0,0,...,1,0,1,1,0,1,0,0,1,0
3,1,0,1,1,1,0,1,0,1,0,...,1,0,1,1,1,1,1,1,1,0
4,1,1,0,0,0,0,0,1,1,1,...,0,1,1,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,1,1,1,1,1,1,1,1,1,1,...,1,0,0,1,1,0,1,0,0,1
241,0,0,0,1,0,0,1,0,1,1,...,0,1,0,0,1,1,0,0,1,0
242,1,1,1,1,1,1,1,1,1,0,...,0,1,0,1,1,1,0,0,1,1
243,1,1,1,1,1,0,1,0,0,0,...,1,1,0,1,1,1,0,0,0,1


Forma pythonica

In [70]:
# Creando df5
df5 = df3.copy()

In [71]:
# lista vacía
frases = []

# comprehension list
frases = [col for col in df5.columns if col.startswith('Según tu')]

In [72]:
# aplicando el bucle, esta vez con una lambda
for frase in frases:
  df5[frase] = df5[frase].apply(lambda x: 1 if x in ['Totalmente verdadero', 'Un poco verdadero'] else 0)

In [73]:
df5[frases] == df4[frases]

,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me preocupa mucho el medio ambiente],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Estudio mucho, doy mi mayor esfuerzo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco el éxito sin importar lo que deba sacrificar],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Trato de vestir sencillo para no presumir],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Creo que la vida se trata de tomar riesgos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco hacer cosas emocionantes para no aburrirme],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [En mi casa, la familia es muy importante, por eso paso mucho tiempo con ellos]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Tener dinero es clave para ser respetado],...,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comer sano y estar en forma],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Vestirse bien es clave para el éxito ],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy sincero con las personas, aunque la verdad les duela]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me preocupo y ayudo a los necesitados],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Invierto mucho tiempo y dinero para aprender cosas nuevas],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta conocer gente y hacer nuevos amigos],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy una persona casera, prefiero estar en casa haciendo lo mío]","Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Prefiero hacer las cosas solo, me estresa trabajar en grupo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta tener muchos amigos y pasar tiempo con ellos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me cuesta concentrarme porque me aburro rápido de las cosas]
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
241,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
242,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
243,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


# Tema 03: Manipulación de datos

## Selección de columnas

In [74]:
df5['Sexo']

,Sexo
0,Hombre
1,Mujer
2,Hombre
3,Mujer
4,Hombre
...,...
240,Hombre
241,Mujer
242,Mujer
243,Hombre


In [75]:
# extraer como df de una sola columna
df5[['Sexo']]

,Sexo
0,Hombre
1,Mujer
2,Hombre
3,Mujer
4,Hombre
...,...
240,Hombre
241,Mujer
242,Mujer
243,Hombre


In [76]:
df5[['Sexo', 'Escribe tu edad exacta']]

,Sexo,Escribe tu edad exacta
0,Hombre,16.0
1,Mujer,16.0
2,Hombre,17.0
3,Mujer,22.0
4,Hombre,16.0
...,...,...
240,Hombre,22.0
241,Mujer,21.0
242,Mujer,22.0
243,Hombre,21.0


In [77]:
# eliminar columnas
df5.drop(columns = 'Marca temporal')

,"¿Estás estudiando en algún colegio, universidad o instituto?",Escribe tu edad exacta,edad2,edadZ,edadR,edadGR,Sexo,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me preocupa mucho el medio ambiente],...,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta conocer gente y hacer nuevos amigos],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy una persona casera, prefiero estar en casa haciendo lo mío]","Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Prefiero hacer las cosas solo, me estresa trabajar en grupo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta tener muchos amigos y pasar tiempo con ellos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me cuesta concentrarme porque me aburro rápido de las cosas],Cuando tenga 30 años me gustaría:,¿Cuánto tiempo has pasado en los últimos 7 días en TikTok? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Instagram? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Facebook? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en YouTube? Si no lo las usado coloca 00:00
0,Sí,16.0,16.0,-1.484925,0.058824,18 o menos,Hombre,0,0,0,...,1,1,1,1,0,"Formar una familia, pues me daría una motivaci...",0:00:00,1:00:00,1:00:00,2:00:00
1,Sí,16.0,16.0,-1.484925,0.058824,18 o menos,Mujer,0,1,1,...,1,1,1,0,0,"Formar una familia, pues me daría una motivaci...",0:50:00,0:00:00,0:00:00,1:00:00
2,Sí,17.0,17.0,-1.231065,0.088235,18 o menos,Hombre,1,0,0,...,1,0,0,1,0,"Formar una familia, pues me daría una motivaci...",8:00:00,4:00:00,0:00:00,8:00:00
3,Sí,22.0,22.0,0.038238,0.235294,Más de 21,Mujer,1,0,1,...,1,1,1,1,0,"Formar una familia, pues me daría una motivaci...",22:35:00,11:21:00,1:57:00,2:40:00
4,Sí,16.0,16.0,-1.484925,0.058824,18 o menos,Hombre,1,1,0,...,1,0,0,1,1,"Estar soltero, muy enfocado en mis logros pers...",7:00:00,1:04:00,0:00:00,3:26:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,Sí,22.0,22.0,0.038238,0.235294,Más de 21,Hombre,1,1,1,...,0,1,0,0,1,"Estar soltero, muy enfocado en mis logros pers...",0:00:00,2:54:00,0:00:00,5:13:00
241,Sí,21.0,21.0,-0.215622,0.205882,19 a 21,Mujer,0,0,0,...,1,0,0,1,0,"Formar una familia, pues me daría una motivaci...",13:00:00,3:25:00,0:39:00,0:00:00
242,Sí,22.0,22.0,0.038238,0.235294,Más de 21,Mujer,1,1,1,...,1,0,0,1,1,"Formar una familia, pues me daría una motivaci...",13:20:00,4:59:00,3:00:00,1:00:00
243,Sí,21.0,21.0,-0.215622,0.205882,19 a 21,Hombre,1,1,1,...,1,0,0,0,1,"Estar soltero, muy enfocado en mis logros pers...",18:16:00,14:42:00,1:08:00,1:00:00


In [78]:
# seleccionar cols que comienzan con...
df5[[col for col in df5.columns if col.startswith('¿Cuánto')]]

,¿Cuánto tiempo has pasado en los últimos 7 días en TikTok? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Instagram? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Facebook? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en YouTube? Si no lo las usado coloca 00:00
0,0:00:00,1:00:00,1:00:00,2:00:00
1,0:50:00,0:00:00,0:00:00,1:00:00
2,8:00:00,4:00:00,0:00:00,8:00:00
3,22:35:00,11:21:00,1:57:00,2:40:00
4,7:00:00,1:04:00,0:00:00,3:26:00
...,...,...,...,...
240,0:00:00,2:54:00,0:00:00,5:13:00
241,13:00:00,3:25:00,0:39:00,0:00:00
242,13:20:00,4:59:00,3:00:00,1:00:00
243,18:16:00,14:42:00,1:08:00,1:00:00


In [79]:
# seleccionar cols que terminan con...
df5[[col for col in df5.columns if col.endswith(']')]]

,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me preocupa mucho el medio ambiente],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Estudio mucho, doy mi mayor esfuerzo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco el éxito sin importar lo que deba sacrificar],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Trato de vestir sencillo para no presumir],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Creo que la vida se trata de tomar riesgos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco hacer cosas emocionantes para no aburrirme],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [En mi casa, la familia es muy importante, por eso paso mucho tiempo con ellos]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Tener dinero es clave para ser respetado],...,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comer sano y estar en forma],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Vestirse bien es clave para el éxito ],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy sincero con las personas, aunque la verdad les duela]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me preocupo y ayudo a los necesitados],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Invierto mucho tiempo y dinero para aprender cosas nuevas],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta conocer gente y hacer nuevos amigos],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy una persona casera, prefiero estar en casa haciendo lo mío]","Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Prefiero hacer las cosas solo, me estresa trabajar en grupo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta tener muchos amigos y pasar tiempo con ellos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me cuesta concentrarme porque me aburro rápido de las cosas]
0,0,0,0,1,1,0,1,0,1,0,...,1,1,1,1,1,1,1,1,1,0
1,0,1,1,0,0,0,1,1,1,0,...,0,1,1,1,0,1,1,1,0,0
2,1,0,0,1,1,1,0,1,0,0,...,1,0,1,1,0,1,0,0,1,0
3,1,0,1,1,1,0,1,0,1,0,...,1,0,1,1,1,1,1,1,1,0
4,1,1,0,0,0,0,0,1,1,1,...,0,1,1,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,1,1,1,1,1,1,1,1,1,1,...,1,0,0,1,1,0,1,0,0,1
241,0,0,0,1,0,0,1,0,1,1,...,0,1,0,0,1,1,0,0,1,0
242,1,1,1,1,1,1,1,1,1,0,...,0,1,0,1,1,1,0,0,1,1
243,1,1,1,1,1,0,1,0,0,0,...,1,1,0,1,1,1,0,0,0,1


In [80]:
# contains
df5.filter(like = 'edad', axis = 1)

,Escribe tu edad exacta,edad2,edadZ,edadR,edadGR
0,16.0,16.0,-1.484925,0.058824,18 o menos
1,16.0,16.0,-1.484925,0.058824,18 o menos
2,17.0,17.0,-1.231065,0.088235,18 o menos
3,22.0,22.0,0.038238,0.235294,Más de 21
4,16.0,16.0,-1.484925,0.058824,18 o menos
...,...,...,...,...,...
240,22.0,22.0,0.038238,0.235294,Más de 21
241,21.0,21.0,-0.215622,0.205882,19 a 21
242,22.0,22.0,0.038238,0.235294,Más de 21
243,21.0,21.0,-0.215622,0.205882,19 a 21


## Selección de filas

In [81]:
# Seleccionar las filas cuando Sexo = 'Mujer'
df5[df5['Sexo'] == 'Mujer'].head()

,Marca temporal,"¿Estás estudiando en algún colegio, universidad o instituto?",Escribe tu edad exacta,edad2,edadZ,edadR,edadGR,Sexo,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status],...,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta conocer gente y hacer nuevos amigos],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy una persona casera, prefiero estar en casa haciendo lo mío]","Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Prefiero hacer las cosas solo, me estresa trabajar en grupo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta tener muchos amigos y pasar tiempo con ellos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me cuesta concentrarme porque me aburro rápido de las cosas],Cuando tenga 30 años me gustaría:,¿Cuánto tiempo has pasado en los últimos 7 días en TikTok? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Instagram? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Facebook? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en YouTube? Si no lo las usado coloca 00:00
1,9/08/2021 12:25:01,Sí,16.0,16.0,-1.484925,0.058824,18 o menos,Mujer,0,1,...,1,1,1,0,0,"Formar una familia, pues me daría una motivaci...",0:50:00,0:00:00,0:00:00,1:00:00
3,9/08/2021 12:28:49,Sí,22.0,22.0,0.038238,0.235294,Más de 21,Mujer,1,0,...,1,1,1,1,0,"Formar una familia, pues me daría una motivaci...",22:35:00,11:21:00,1:57:00,2:40:00
5,9/08/2021 12:34:23,Sí,18.0,18.0,-0.977204,0.117647,18 o menos,Mujer,1,0,...,1,0,0,1,0,"Formar una familia, pues me daría una motivaci...",1:16:00,1:00:00,14:00:00,3:00:00
6,9/08/2021 12:35:46,Sí,14.0,14.0,-1.992646,0.000000,18 o menos,Mujer,1,1,...,1,1,0,1,1,"Formar una familia, pues me daría una motivaci...",8:00:00,12:00:00,5:00:00,2:00:00
8,9/08/2021 12:40:30,Sí,22.0,22.0,0.038238,0.235294,Más de 21,Mujer,0,1,...,1,1,0,1,1,"Formar una familia, pues me daría una motivaci...",0:51:00,0:55:00,0:27:00,0:35:00


In [82]:
# Seleccionar las filas cuando Sexo no es igual a 'Hombre'
df5[df5['Sexo'] != 'Hombre'].head()

,Marca temporal,"¿Estás estudiando en algún colegio, universidad o instituto?",Escribe tu edad exacta,edad2,edadZ,edadR,edadGR,Sexo,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status],...,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta conocer gente y hacer nuevos amigos],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy una persona casera, prefiero estar en casa haciendo lo mío]","Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Prefiero hacer las cosas solo, me estresa trabajar en grupo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta tener muchos amigos y pasar tiempo con ellos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me cuesta concentrarme porque me aburro rápido de las cosas],Cuando tenga 30 años me gustaría:,¿Cuánto tiempo has pasado en los últimos 7 días en TikTok? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Instagram? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Facebook? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en YouTube? Si no lo las usado coloca 00:00
1,9/08/2021 12:25:01,Sí,16.0,16.0,-1.484925,0.058824,18 o menos,Mujer,0,1,...,1,1,1,0,0,"Formar una familia, pues me daría una motivaci...",0:50:00,0:00:00,0:00:00,1:00:00
3,9/08/2021 12:28:49,Sí,22.0,22.0,0.038238,0.235294,Más de 21,Mujer,1,0,...,1,1,1,1,0,"Formar una familia, pues me daría una motivaci...",22:35:00,11:21:00,1:57:00,2:40:00
5,9/08/2021 12:34:23,Sí,18.0,18.0,-0.977204,0.117647,18 o menos,Mujer,1,0,...,1,0,0,1,0,"Formar una familia, pues me daría una motivaci...",1:16:00,1:00:00,14:00:00,3:00:00
6,9/08/2021 12:35:46,Sí,14.0,14.0,-1.992646,0.000000,18 o menos,Mujer,1,1,...,1,1,0,1,1,"Formar una familia, pues me daría una motivaci...",8:00:00,12:00:00,5:00:00,2:00:00
8,9/08/2021 12:40:30,Sí,22.0,22.0,0.038238,0.235294,Más de 21,Mujer,0,1,...,1,1,0,1,1,"Formar una familia, pues me daría una motivaci...",0:51:00,0:55:00,0:27:00,0:35:00


In [83]:
# Seleccionar filas cuando edad2 es mayor o igual a 20
df5[df5['edad2'] >= 20].head()

,Marca temporal,"¿Estás estudiando en algún colegio, universidad o instituto?",Escribe tu edad exacta,edad2,edadZ,edadR,edadGR,Sexo,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status],...,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta conocer gente y hacer nuevos amigos],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy una persona casera, prefiero estar en casa haciendo lo mío]","Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Prefiero hacer las cosas solo, me estresa trabajar en grupo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta tener muchos amigos y pasar tiempo con ellos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me cuesta concentrarme porque me aburro rápido de las cosas],Cuando tenga 30 años me gustaría:,¿Cuánto tiempo has pasado en los últimos 7 días en TikTok? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Instagram? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Facebook? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en YouTube? Si no lo las usado coloca 00:00
3,9/08/2021 12:28:49,Sí,22.0,22.0,0.038238,0.235294,Más de 21,Mujer,1,0,...,1,1,1,1,0,"Formar una familia, pues me daría una motivaci...",22:35:00,11:21:00,1:57:00,2:40:00
7,9/08/2021 12:38:32,Sí,20.0,20.0,-0.469483,0.176471,19 a 21,Hombre,1,0,...,1,0,1,1,1,"Estar soltero, muy enfocado en mis logros pers...",13:11:00,0:43:00,4:21:00,6:17:00
8,9/08/2021 12:40:30,Sí,22.0,22.0,0.038238,0.235294,Más de 21,Mujer,0,1,...,1,1,0,1,1,"Formar una familia, pues me daría una motivaci...",0:51:00,0:55:00,0:27:00,0:35:00
9,9/08/2021 12:41:37,Sí,21.0,21.0,-0.215622,0.205882,19 a 21,Mujer,0,0,...,1,0,0,1,1,"Formar una familia, pues me daría una motivaci...",22:34:00,10:50:00,3:27:00,11:54:00
11,9/08/2021 12:43:08,Sí,22.0,22.0,0.038238,0.235294,Más de 21,Mujer,0,1,...,0,1,1,1,1,"Formar una familia, pues me daría una motivaci...",2:47:00,4:12:00,3:45:00,3:45:00


In [84]:
# Seleccionar filas cuando edad2 está entre 18 y 21
df5[(df5['edad2'] >= 18) & (df5['edad2'] <= 21)].head()

,Marca temporal,"¿Estás estudiando en algún colegio, universidad o instituto?",Escribe tu edad exacta,edad2,edadZ,edadR,edadGR,Sexo,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status],...,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta conocer gente y hacer nuevos amigos],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy una persona casera, prefiero estar en casa haciendo lo mío]","Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Prefiero hacer las cosas solo, me estresa trabajar en grupo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta tener muchos amigos y pasar tiempo con ellos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me cuesta concentrarme porque me aburro rápido de las cosas],Cuando tenga 30 años me gustaría:,¿Cuánto tiempo has pasado en los últimos 7 días en TikTok? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Instagram? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Facebook? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en YouTube? Si no lo las usado coloca 00:00
5,9/08/2021 12:34:23,Sí,18.0,18.0,-0.977204,0.117647,18 o menos,Mujer,1,0,...,1,0,0,1,0,"Formar una familia, pues me daría una motivaci...",1:16:00,1:00:00,14:00:00,3:00:00
7,9/08/2021 12:38:32,Sí,20.0,20.0,-0.469483,0.176471,19 a 21,Hombre,1,0,...,1,0,1,1,1,"Estar soltero, muy enfocado en mis logros pers...",13:11:00,0:43:00,4:21:00,6:17:00
9,9/08/2021 12:41:37,Sí,21.0,21.0,-0.215622,0.205882,19 a 21,Mujer,0,0,...,1,0,0,1,1,"Formar una familia, pues me daría una motivaci...",22:34:00,10:50:00,3:27:00,11:54:00
12,9/08/2021 12:43:25,Sí,21.0,21.0,-0.215622,0.205882,19 a 21,Mujer,1,0,...,1,0,0,1,0,"Estar soltero, muy enfocado en mis logros pers...",3:00:00,2:00:00,1:00:00,2:00:00
13,9/08/2021 12:49:53,Sí,18.0,18.0,-0.977204,0.117647,18 o menos,Mujer,1,1,...,1,0,0,1,1,"Formar una familia, pues me daría una motivaci...",0:00:00,0:10:00,10:00:00,5:00:00


In [85]:
# Seleccionar filas cuando edad está entre 18 y 21 y cuando sexo = 'Mujer'
df5[
    (df5['edad2'] >= 18) &
    (df5['edad2'] <= 21) &
    (df5['Sexo'] == 'Mujer')
].head()

,Marca temporal,"¿Estás estudiando en algún colegio, universidad o instituto?",Escribe tu edad exacta,edad2,edadZ,edadR,edadGR,Sexo,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status],...,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta conocer gente y hacer nuevos amigos],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy una persona casera, prefiero estar en casa haciendo lo mío]","Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Prefiero hacer las cosas solo, me estresa trabajar en grupo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta tener muchos amigos y pasar tiempo con ellos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me cuesta concentrarme porque me aburro rápido de las cosas],Cuando tenga 30 años me gustaría:,¿Cuánto tiempo has pasado en los últimos 7 días en TikTok? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Instagram? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Facebook? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en YouTube? Si no lo las usado coloca 00:00
5,9/08/2021 12:34:23,Sí,18.0,18.0,-0.977204,0.117647,18 o menos,Mujer,1,0,...,1,0,0,1,0,"Formar una familia, pues me daría una motivaci...",1:16:00,1:00:00,14:00:00,3:00:00
9,9/08/2021 12:41:37,Sí,21.0,21.0,-0.215622,0.205882,19 a 21,Mujer,0,0,...,1,0,0,1,1,"Formar una familia, pues me daría una motivaci...",22:34:00,10:50:00,3:27:00,11:54:00
12,9/08/2021 12:43:25,Sí,21.0,21.0,-0.215622,0.205882,19 a 21,Mujer,1,0,...,1,0,0,1,0,"Estar soltero, muy enfocado en mis logros pers...",3:00:00,2:00:00,1:00:00,2:00:00
13,9/08/2021 12:49:53,Sí,18.0,18.0,-0.977204,0.117647,18 o menos,Mujer,1,1,...,1,0,0,1,1,"Formar una familia, pues me daría una motivaci...",0:00:00,0:10:00,10:00:00,5:00:00
27,9/08/2021 16:07:23,Sí,18.0,18.0,-0.977204,0.117647,18 o menos,Mujer,1,0,...,1,0,1,1,1,"Estar soltero, muy enfocado en mis logros pers...",10:44:00,9:08:00,3:41:00,0:29:00


In [86]:
df2[
    (df2['edad2'] > 20) &
    (df2['Sexo'] == 'Hombre') &
    (df2.iloc[:,8].isin(['Totalmente verdadero', 'Un poco verdadero']))
].head()

,Marca temporal,"¿Estás estudiando en algún colegio, universidad o instituto?",Escribe tu edad exacta,edad2,Sexo,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me preocupa mucho el medio ambiente],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Estudio mucho, doy mi mayor esfuerzo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco el éxito sin importar lo que deba sacrificar],...,Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta conocer gente y hacer nuevos amigos],"Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Soy una persona casera, prefiero estar en casa haciendo lo mío]","Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Prefiero hacer las cosas solo, me estresa trabajar en grupo]",Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta tener muchos amigos y pasar tiempo con ellos],Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me cuesta concentrarme porque me aburro rápido de las cosas],Cuando tenga 30 años me gustaría:,¿Cuánto tiempo has pasado en los últimos 7 días en TikTok? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Instagram? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en Facebook? Si no lo las usado coloca 00:00,¿Cuánto tiempo has pasado en los últimos 7 días en YouTube? Si no lo las usado coloca 00:00
23,9/08/2021 14:06:44,Sí,25.0,25.0,Hombre,Totalmente verdadero,Totalmente falso,Totalmente verdadero,Totalmente verdadero,Un poco verdadero,...,No lo sé,Totalmente verdadero,No lo sé,Un poco falso,Un poco falso,"Formar una familia, pues me daría una motivaci...",0:19:00,4:00:00,1:00:00,3:50:00
46,11/08/2021 11:43:40,Sí,21.0,21.0,Hombre,Totalmente verdadero,Totalmente verdadero,Un poco falso,Un poco verdadero,Un poco verdadero,...,Un poco verdadero,Un poco verdadero,Un poco verdadero,Un poco falso,Un poco verdadero,"Estar soltero, muy enfocado en mis logros pers...",0:00:00,21:37:00,16:09:00,14:00:00
50,11/08/2021 11:44:59,Sí,21.0,21.0,Hombre,Totalmente verdadero,No lo sé,Un poco verdadero,Totalmente verdadero,Un poco verdadero,...,Totalmente verdadero,Un poco verdadero,No lo sé,Un poco verdadero,Totalmente verdadero,"Formar una familia, pues me daría una motivaci...",8:52:00,1:58:00,0:46:00,1:10:00
52,11/08/2021 11:46:19,Sí,25.0,25.0,Hombre,Totalmente verdadero,Totalmente falso,Un poco verdadero,Un poco verdadero,Un poco verdadero,...,Totalmente verdadero,Un poco falso,Totalmente falso,Totalmente verdadero,Totalmente verdadero,"Estar soltero, muy enfocado en mis logros pers...",3:00:00,6:00:00,0:00:00,1:00:00
57,11/08/2021 11:53:02,Sí,22.0,22.0,Hombre,Un poco verdadero,No lo sé,Un poco verdadero,Totalmente verdadero,No lo sé,...,Un poco verdadero,No lo sé,Un poco falso,Totalmente verdadero,Un poco falso,"Formar una familia, pues me daría una motivaci...",0:00:00,5:00:00,10:00:00,3:00:00


## Renombrando columnas

In [87]:
df6 = df5.copy()

In [88]:
df6.columns

Index(['Marca temporal',
       '¿Estás estudiando en algún colegio, universidad o instituto?',
       'Escribe tu edad exacta', 'edad2', 'edadZ', 'edadR', 'edadGR', 'Sexo',
       'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [No discrimino y trato a todos por igual]',
       'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me gusta comprar marcas que representen mi status]',
       'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Me preocupa mucho el medio ambiente]',
       'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Estudio mucho, doy mi mayor esfuerzo]',
       'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Busco el éxito sin importar lo que deba sacrificar]',
       'Según tu forma de ser ¿Cuál de las siguientes frases te describe mejor: [Trato de vestir sencillo para no presumir]',
       'Según tu forma de ser ¿Cuál de las siguientes frases te de

APPS

**Paso 1:** Crear una lista con los nuevos nombres

In [89]:
apps = ['TikTok', 'Instagram', 'Facebook', 'YouTube']

**Paso 2:** Crear dict que contenga los nombres antiguos como *keys* y nuevos como *values*

In [90]:
apps_dict = dict(zip(
    [col for col in df6.columns if col.startswith('¿Cuánto')],
    apps
))

In [91]:
apps_dict

{'¿Cuánto tiempo has pasado en los últimos 7 días en TikTok? Si no lo las usado coloca 00:00': 'TikTok',
 '¿Cuánto tiempo has pasado en los últimos 7 días en Instagram? Si no lo las usado coloca 00:00': 'Instagram',
 '¿Cuánto tiempo has pasado en los últimos 7 días en Facebook? Si no lo las usado coloca 00:00': 'Facebook',
 '¿Cuánto tiempo has pasado en los últimos 7 días en YouTube? Si no lo las usado coloca 00:00': 'YouTube'}

**Paso 3:** Renombrando

In [92]:
df6.rename(apps_dict, axis = 1, inplace = True)

In [93]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
Index: 239 entries, 0 to 244
Data columns (total 37 columns):
 #   Column                                                                                                                                                   Non-Null Count  Dtype   
---  ------                                                                                                                                                   --------------  -----   
 0   Marca temporal                                                                                                                                           239 non-null    object  
 1   ¿Estás estudiando en algún colegio, universidad o instituto?                                                                                             239 non-null    object  
 2   Escribe tu edad exacta                                                                                                                                   239 non-nu

Frases

**Paso 0:** Convertir la lista en Pandas Series

In [94]:
type(frases)

list

In [95]:
frases2 = pd.Series(frases)

In [96]:
type(frases2)

pandas.core.series.Series

**Paso 1:** Crear una lista con los nuevos nombres

In [97]:
frases2 = frases2.str.extract('\[(.+)\]')[0]

In [98]:
frases2

,0
0,No discrimino y trato a todos por igual
1,Me gusta comprar marcas que representen mi status
2,Me preocupa mucho el medio ambiente
3,"Estudio mucho, doy mi mayor esfuerzo"
4,Busco el éxito sin importar lo que deba sacrif...
5,Trato de vestir sencillo para no presumir
6,Creo que la vida se trata de tomar riesgos
7,Busco hacer cosas emocionantes para no aburrirme
8,"En mi casa, la familia es muy importante, por ..."
9,Tener dinero es clave para ser respetado


**Paso 2:** Crear dict que contenga los nombres antiguos como *keys* y nuevos como *values*

In [99]:
frases_dict = dict(zip(frases, frases2))

**Pase 3:** Renombrando

In [100]:
df6.rename(frases_dict, axis=1, inplace=True)

In [101]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
Index: 239 entries, 0 to 244
Data columns (total 37 columns):
 #   Column                                                                         Non-Null Count  Dtype   
---  ------                                                                         --------------  -----   
 0   Marca temporal                                                                 239 non-null    object  
 1   ¿Estás estudiando en algún colegio, universidad o instituto?                   239 non-null    object  
 2   Escribe tu edad exacta                                                         239 non-null    float64 
 3   edad2                                                                          239 non-null    float64 
 4   edadZ                                                                          239 non-null    float64 
 5   edadR                                                                          239 non-null    float64 
 6   edadGR                 

## Pivotado

### Pivot longer

In [102]:
apps

['TikTok', 'Instagram', 'Facebook', 'YouTube']

In [103]:
df7 = df6.melt(
    id_vars = ['Marca temporal', 'Sexo', 'edad2'],
    value_vars = apps,
    var_name = 'app',
    value_name = 'time'
)

In [104]:
df7.head()

,Marca temporal,Sexo,edad2,app,time
0,9/08/2021 12:23:46,Hombre,16.0,TikTok,0:00:00
1,9/08/2021 12:25:01,Mujer,16.0,TikTok,0:50:00
2,9/08/2021 12:26:31,Hombre,17.0,TikTok,8:00:00
3,9/08/2021 12:28:49,Mujer,22.0,TikTok,22:35:00
4,9/08/2021 12:31:43,Hombre,16.0,TikTok,7:00:00


### Pivot wider

In [105]:
df8 = df7.pivot(
    index = ['Marca temporal', 'Sexo', 'edad2'],
    columns = 'app',
    values = 'time'
).reset_index()

In [106]:
df8.head()

app,Marca temporal,Sexo,edad2,Facebook,Instagram,TikTok,YouTube
0,10/01/2023 18:16:15,Mujer,25.0,0:00:00,6:00:00,0:20:00,3:00:00
1,10/01/2023 18:16:58,Mujer,21.0,5:00:00,6:00:00,3:00:00,2:00:00
2,10/01/2023 18:17:15,Hombre,22.0,5:00:00,1:00:00,3:00:00,3:00:00
3,10/01/2023 18:17:22,Mujer,22.0,1:50:00,6:33:00,7:52:00,0:54:00
4,10/01/2023 18:17:33,Mujer,25.0,0:37:00,2:15:00,1:28:00,0:00:00


# Tema 04: Outliers

In [108]:
df7.head()

,Marca temporal,Sexo,edad2,app,time
0,9/08/2021 12:23:46,Hombre,16.0,TikTok,0:00:00
1,9/08/2021 12:25:01,Mujer,16.0,TikTok,0:50:00
2,9/08/2021 12:26:31,Hombre,17.0,TikTok,8:00:00
3,9/08/2021 12:28:49,Mujer,22.0,TikTok,22:35:00
4,9/08/2021 12:31:43,Hombre,16.0,TikTok,7:00:00


In [109]:
df7['time'].dtype

dtype('O')

Transformando horas a número

In [114]:
# split() separa texto
'22:35:00'.split(':')

'00'

In [115]:
# función conversora
def tiempo_a_num(x):
  x = x.split(':')
  return int(x[0]) + int(x[1])/60 + int(x[2])/3600

In [116]:
tiempo_a_num('22:35:00')

22.583333333333332

In [118]:
# aplicar tiempo_a_num() a la columna 'time' del df7
df7['time'] = df7['time'].apply(tiempo_a_num)

In [120]:
df7['time'].head()

,time
0,0.000000
1,0.833333
2,8.000000
3,22.583333
4,7.000000


## Boxplots

In [121]:
import plotly.express as px

In [124]:
boxplot_01 = px.box(df7, y = 'time')
boxplot_01.show()

In [133]:
px.box(
    df7,
    x = 'app',
    y = 'time',
    color = 'app'
).update_layout({
    'plot_bgcolor': 'white',
    'showlegend': False
}).update_traces({
    'fillcolor': 'white',
    'line': {'width': 2}
})

## Identificando outliers en el df

In [135]:
df9 = df7.copy()

In [138]:
condiciones = (
    ((df9['app'] == 'TikTok')    & (df9['time'] > 18)) |
    ((df9['app'] == 'Instagram') & (df9['time'] > 12)) |
    ((df9['app'] == 'Facebook')  & (df9['time'] > 10)) |
    ((df9['app'] == 'YouTube')   & (df9['time'] > 9))
)

In [139]:
df9['outlier'] = np.where(condiciones, 'outlier', 'no outlier')

In [140]:
df9

,Marca temporal,Sexo,edad2,app,time,outlier
0,9/08/2021 12:23:46,Hombre,16.0,TikTok,0.000000,no outlier
1,9/08/2021 12:25:01,Mujer,16.0,TikTok,0.833333,no outlier
2,9/08/2021 12:26:31,Hombre,17.0,TikTok,8.000000,no outlier
3,9/08/2021 12:28:49,Mujer,22.0,TikTok,22.583333,outlier
4,9/08/2021 12:31:43,Hombre,16.0,TikTok,7.000000,no outlier
...,...,...,...,...,...,...
951,8/02/2025 10:43:39,Hombre,22.0,YouTube,5.216667,no outlier
952,8/02/2025 10:44:18,Mujer,21.0,YouTube,0.000000,no outlier
953,8/02/2025 10:44:33,Mujer,22.0,YouTube,1.000000,no outlier
954,8/02/2025 10:44:36,Hombre,21.0,YouTube,1.000000,no outlier


In [ ]:
# creando una columna con el promedio
df9['promedio'] = df9.groupby('app')['time'].transform('mean')

In [150]:
df9

,Marca temporal,Sexo,edad2,app,time,outlier,promedio
0,9/08/2021 12:23:46,Hombre,16.0,TikTok,0.000000,no outlier,5.211715
1,9/08/2021 12:25:01,Mujer,16.0,TikTok,0.833333,no outlier,5.211715
2,9/08/2021 12:26:31,Hombre,17.0,TikTok,8.000000,no outlier,5.211715
3,9/08/2021 12:28:49,Mujer,22.0,TikTok,22.583333,outlier,5.211715
4,9/08/2021 12:31:43,Hombre,16.0,TikTok,7.000000,no outlier,5.211715
...,...,...,...,...,...,...,...
951,8/02/2025 10:43:39,Hombre,22.0,YouTube,5.216667,no outlier,3.265690
952,8/02/2025 10:44:18,Mujer,21.0,YouTube,0.000000,no outlier,3.265690
953,8/02/2025 10:44:33,Mujer,22.0,YouTube,1.000000,no outlier,3.265690
954,8/02/2025 10:44:36,Hombre,21.0,YouTube,1.000000,no outlier,3.265690


In [151]:
df9['time2'] = np.where(
    df9['outlier'] == 'outlier',
    df9['promedio'],
    df9['time']
)

In [152]:
df9

,Marca temporal,Sexo,edad2,app,time,outlier,promedio,time2
0,9/08/2021 12:23:46,Hombre,16.0,TikTok,0.000000,no outlier,5.211715,0.000000
1,9/08/2021 12:25:01,Mujer,16.0,TikTok,0.833333,no outlier,5.211715,0.833333
2,9/08/2021 12:26:31,Hombre,17.0,TikTok,8.000000,no outlier,5.211715,8.000000
3,9/08/2021 12:28:49,Mujer,22.0,TikTok,22.583333,outlier,5.211715,5.211715
4,9/08/2021 12:31:43,Hombre,16.0,TikTok,7.000000,no outlier,5.211715,7.000000
...,...,...,...,...,...,...,...,...
951,8/02/2025 10:43:39,Hombre,22.0,YouTube,5.216667,no outlier,3.265690,5.216667
952,8/02/2025 10:44:18,Mujer,21.0,YouTube,0.000000,no outlier,3.265690,0.000000
953,8/02/2025 10:44:33,Mujer,22.0,YouTube,1.000000,no outlier,3.265690,1.000000
954,8/02/2025 10:44:36,Hombre,21.0,YouTube,1.000000,no outlier,3.265690,1.000000


# Tema 05: Tablas

## Simples

In [155]:
# conteo
df9['outlier'].value_counts()

,count
outlier,
no outlier,895
outlier,61


In [156]:
# proporciones
df9['outlier'].value_counts(normalize = True)

,proportion
outlier,
no outlier,0.936192
outlier,0.063808


## Tablas cruzadas

In [162]:
df9 \
  .groupby(['app', 'Sexo']) \
  .agg(
      conteo = ('time2', 'size'),
      promedio_time_1 = ('time', 'mean'),
      promedio_time_2 = ('time2', 'mean'),
      desviacion_time = ('time', 'std'),
      desviacion_time2 = ('time2', 'std')
  ) \
  .reset_index()

,app,Sexo,conteo,promedio_time_1,promedio_time_2,desviacion_time,desviacion_time2
0,Facebook,Hombre,108,3.095216,2.513501,4.096159,2.613365
1,Facebook,Mujer,131,2.993639,2.432727,3.841672,2.605630
2,Instagram,Hombre,108,3.724691,2.952056,4.434169,2.675969
3,Instagram,Mujer,131,4.405980,3.530151,4.513879,2.865217
4,TikTok,Hombre,108,5.083179,4.492409,5.438588,4.408389
5,TikTok,Mujer,131,5.317684,4.566567,5.718346,4.448446
6,YouTube,Hombre,108,4.424846,2.656800,5.394609,2.245938
7,YouTube,Mujer,131,2.310051,1.309340,4.624042,1.647244
